In [1]:
!pip install -U bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 19.8 MB/s eta 0:00:00


In [2]:
!pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 58.3 MB/s eta 0:00:00


In [7]:
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu # Note: VideoReader and decord.cpu are not used in this image example
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

# Define build_transform and dynamic_preprocess functions
def build_transform(input_size=448):
    """Builds a transformation pipeline for image preprocessing."""
    transform = T.Compose([
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform


def dynamic_preprocess(image, image_size=448, use_thumbnail=True, max_num=12):
    """Dynamically preprocesses the image."""
    # Placeholder for dynamic preprocessing logic.
    # This example simply returns the original image in a list.
    # Replace with actual implementation if needed.
    return [image]


def load_image(image_file, input_size=448, max_num=None):
    """Loads an image and preprocesses it.
    If 'max_num' is not None, it is passed to dynamic_preprocess, otherwise a default of 12 is used."""
    image = Image.open(image_file).convert('RGB')
    transform_fn = build_transform(input_size=input_size) # Renamed to avoid conflict with T.transform

    # Determine max_num for dynamic_preprocess
    # The original logic had a slight confusion. If max_num is None, use 12.
    # If max_num is provided (not None), use that provided value.
    current_max_num = max_num if max_num is not None else 12

    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=current_max_num)

    pixel_values_list = [transform_fn(img) for img in images] # Renamed 'image' in loop to 'img'
    pixel_values_tensor = torch.stack(pixel_values_list)
    return pixel_values_tensor

# Load the model and tokenizer
path = "OpenGVLab/InternVL2_5-1B"

# Ensure CUDA is available, otherwise, this loading config might error or behave unexpectedly
if not torch.cuda.is_available():
    print("Warning: CUDA is not available. load_in_8bit=True might not work as expected or might default to CPU.")
    # If you intend to run on CPU, you might need to change model loading parameters:
    # model = AutoModel.from_pretrained(path, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, trust_remote_code=True).eval()
    # And ensure pixel_values remain on CPU.
    # For this fix, we assume CUDA is available as per the error message.

model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=True,  # This usually moves the model to GPU if available
    low_cpu_mem_usage=True,
    use_flash_attn=True, # Requires compatible GPU and toolkit
    trust_remote_code=True
).eval()

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

# Define image path
# Make sure this path is correct and the image exists.
# For example, if running in Colab, upload the image or use a sample path.
# image_path = '/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg'
# Using a placeholder for testing if the image is not available
try:
    Image.open('/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg')
    image_path = '/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg'
except FileNotFoundError:
    print("Warning: Image file not found. Using a dummy black image.")
    dummy_image = Image.new('RGB', (100, 100), color = 'black')
    dummy_image.save("dummy_image.jpg")
    image_path = "dummy_image.jpg"


# Load image, convert to bfloat16, and move to the model's device
pixel_values = load_image(image_path) # max_num will default to 12 as per function definition
pixel_values = pixel_values.to(torch.bfloat16).to(model.device) # Move to the same device as the model

# Define generation configuration
generation_config = dict(
    num_beams=3, # Example, common for generation
    max_new_tokens=1024,
    do_sample=True,       # Enable sampling to use the temperature parameter
    temperature=0.2       # Lower temperature for less random, more focused output
)

# Single-image single-round conversation
question = '<image>\nYou are an expert Optical Character Recognition (OCR) assistant.Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image. Transcribe the text exactly as it appears on the document, character by character. You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image. Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.'

# Perform chat
try:
    response = model.chat(tokenizer, pixel_values, question, generation_config)
    print(f'User: {question}\nAssistant: {response}')
except RuntimeError as e:
    print(f"An error occurred during model.chat: {e}")
    if "out of memory" in str(e).lower():
        print("CUDA out of memory. Try reducing batch size, image resolution, or using a smaller model if possible.")
    elif "expected all tensors to be on the same device" in str(e).lower():
         print("Device mismatch error persists. Model device:", model.device, "Pixel values device:", pixel_values.device)
    else:
        # For other RuntimeErrors, you might want to inspect model.device and pixel_values.device
        print(f"Model device: {model.device}")
        if 'pixel_values' in locals():
             print(f"Pixel_values device: {pixel_values.device}")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


FlashAttention2 is not installed.


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: <image>
You are an expert Optical Character Recognition (OCR) assistant.Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image. Transcribe the text exactly as it appears on the document, character by character. You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image. Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.
Assistant: 1234568
CALIFORNIA
DRIVER LICENSE
ID
I1234568
EXP 08/31/2015
LN CARDHOLDER
FN IMA
2570 24TH STREET
SACRAMENTO, CA 95181
DOB 08/31/1977
RST NONE
SEX F
HAIR BRN
EYES BRN
DD 09/30/2010
0831977


In [6]:
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu # Note: VideoReader and decord.cpu are not used in this image example
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

# Define build_transform and dynamic_preprocess functions
def build_transform(input_size=448):
    """Builds a transformation pipeline for image preprocessing."""
    transform = T.Compose([
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform


def dynamic_preprocess(image, image_size=448, use_thumbnail=True, max_num=12):
    """Dynamically preprocesses the image."""
    # Placeholder for dynamic preprocessing logic.
    # This example simply returns the original image in a list.
    # Replace with actual implementation if needed.
    return [image]


def load_image(image_file, input_size=448, max_num=None):
    """Loads an image and preprocesses it.
    If 'max_num' is not None, it is passed to dynamic_preprocess, otherwise a default of 12 is used."""
    image = Image.open(image_file).convert('RGB')
    transform_fn = build_transform(input_size=input_size) # Renamed to avoid conflict with T.transform

    # Determine max_num for dynamic_preprocess
    # The original logic had a slight confusion. If max_num is None, use 12.
    # If max_num is provided (not None), use that provided value.
    current_max_num = max_num if max_num is not None else 12

    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=current_max_num)

    pixel_values_list = [transform_fn(img) for img in images] # Renamed 'image' in loop to 'img'
    pixel_values_tensor = torch.stack(pixel_values_list)
    return pixel_values_tensor

# Load the model and tokenizer
path = "OpenGVLab/InternVL2_5-2B"

# Ensure CUDA is available, otherwise, this loading config might error or behave unexpectedly
if not torch.cuda.is_available():
    print("Warning: CUDA is not available. load_in_8bit=True might not work as expected or might default to CPU.")
    # If you intend to run on CPU, you might need to change model loading parameters:
    # model = AutoModel.from_pretrained(path, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, trust_remote_code=True).eval()
    # And ensure pixel_values remain on CPU.
    # For this fix, we assume CUDA is available as per the error message.

model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=True,  # This usually moves the model to GPU if available
    low_cpu_mem_usage=True,
    use_flash_attn=True, # Requires compatible GPU and toolkit
    trust_remote_code=True
).eval()

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

# Define image path
# Make sure this path is correct and the image exists.
# For example, if running in Colab, upload the image or use a sample path.
# image_path = '/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg'
# Using a placeholder for testing if the image is not available
try:
    Image.open('/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg')
    image_path = '/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg'
except FileNotFoundError:
    print("Warning: Image file not found. Using a dummy black image.")
    dummy_image = Image.new('RGB', (100, 100), color = 'black')
    dummy_image.save("dummy_image.jpg")
    image_path = "dummy_image.jpg"


# Load image, convert to bfloat16, and move to the model's device
pixel_values = load_image(image_path) # max_num will default to 12 as per function definition
pixel_values = pixel_values.to(torch.bfloat16).to(model.device) # Move to the same device as the model

# Define generation configuration
generation_config = dict(
    num_beams=3,          # You can keep beam search or set to 1 for pure sampling
    max_new_tokens=1024,
    do_sample=True,       # Enable sampling to use the temperature parameter
    temperature=0.2       # Lower temperature for less random, more focused output
)

# Single-image single-round conversation
question = '<image>\nYou are an expert Optical Character Recognition (OCR) assistant.Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image. Transcribe the text exactly as it appears on the document, character by character. You MUST ONLY output the extracted text. Do NOT add any key, introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not on the image. Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.'

# Perform chat
try:
    response = model.chat(tokenizer, pixel_values, question, generation_config)
    print(f'User: {question}\nAssistant: {response}')
except RuntimeError as e:
    print(f"An error occurred during model.chat: {e}")
    if "out of memory" in str(e).lower():
        print("CUDA out of memory. Try reducing batch size, image resolution, or using a smaller model if possible.")
    elif "expected all tensors to be on the same device" in str(e).lower():
         print("Device mismatch error persists. Model device:", model.device, "Pixel values device:", pixel_values.device)
    else:
        # For other RuntimeErrors, you might want to inspect model.device and pixel_values.device
        print(f"Model device: {model.device}")
        if 'pixel_values' in locals():
             print(f"Pixel_values device: {pixel_values.device}")

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
InternLM2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classe

FlashAttention2 is not installed.


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


User: <image>
You are an expert Optical Character Recognition (OCR) assistant.Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image. Transcribe the text exactly as it appears on the document, character by character. You MUST ONLY output the extracted text. Do NOT add any key, introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not on the image. Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.
Assistant: I1234568
EXPI 08/31/2015
LN CARDHOLDER
IMA
2570 24TH STREET
SACRAMENTO, CA 95818
08/31/1977
DOB 08/31/1977
RSTR NONE
SEX F
HGT 5-05
WGT 125 lb
EYES BRN
ISS 09/30/2010


Stage 02

In [2]:
# test.py
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
import traceback # For detailed error reporting
import time      # For measuring execution time

# --- Model and Tokenizer Loading ---
print("Loading model and tokenizer...")
try:
    model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6-int4', trust_remote_code=True, device_map='auto')
    tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6-int4', trust_remote_code=True)
    model.eval()
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    traceback.print_exc()
    exit()

Loading model and tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and tokenizer loaded successfully.


In [4]:
# --- Record Start Time ---
start_time = time.time()

# --- OCR Instruction Prompt ---
ocr_instruction_prompt = """OCR ACCURATELY AND SHARE EXACTLY HOW THE TEXTS APPEAR ON IMAGE RETAINING SAME STRUCTURE. SHARE TEXTS WITHOUT ADDITIONAL TITLE, KEY, SYMBOLS, COMMENTS.
"""

# --- Image Loading ---
img_path = "/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg" # Ensure this path is correct

try:
    image = Image.open(img_path).convert('RGB')
    print(f"Image loaded successfully from {img_path}")
except FileNotFoundError:
    print(f"Error: Image file not found at {img_path}")
    print("Please make sure the image is uploaded to Colab and the path is correct.")
    exit()
except Exception as e:
    print(f"An error occurred while opening the image: {e}")
    traceback.print_exc()
    exit()

# --- Prepare messages for the model ---
# The user message now contains the image AND the detailed OCR instruction.
msgs = [
    {'role': 'user', 'content': [image, ocr_instruction_prompt]}
]

# --- Streaming OCR Output ---
print("\n--- Streaming OCR Output ---")
generated_text_streamed = ""
try:
    res_stream = model.chat(
        image=None, # Image is in msgs
        msgs=msgs,
        tokenizer=tokenizer,
        sampling=True,
        temperature=0.07,
        stream=True
    )

    print("OCR Output: ", end='')
    for new_text in res_stream:
        generated_text_streamed += new_text
        print(new_text, flush=True, end='')
    print() # For a final newline after streaming is done
    # print(f"\nFull streamed text (for debugging if needed): {generated_text_streamed}")
except Exception as e:
    print(f"\nError during streaming model chat: {e}")
    print(f"Type of exception: {type(e)}")
    print("Traceback:")
    traceback.print_exc()

# --- Calculate and Print Execution Time ---
end_time = time.time()
total_time = end_time - start_time
print(f"\n--- OCR Bot Script Finished ---")
print(f"Total execution time: {total_time:.2f} seconds")

Image loaded successfully from /content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg

--- Streaming OCR Output ---
OCR Output: Title: CALIFORNIA DRIVER LICENSE

ID Number: [1234568]
Expiration Date: EXP 08/31/2015
License Status: CLASS C END NONE
Last Name: CARDHOLDER LN
First Name Initial: IMA FN
Address: 2570 24TH STREET SACRAMENTO, CA 95818
Date of Birth: DOB 08/31/1977
Restrictions: RSTR NONE
Donor Information: DONOR

Sex: SEX F
Hair Color: HAIR BRN
Height: HGT 5'-05"
Weight: WGT 125 lb
Eye Color: EYES BRN
Date of Issue: DD 09/30/2010

--- OCR Bot Script Finished ---
Total execution time: 18.42 seconds


# Another Model

In [3]:
!pip install -U pyvips libvips

ERROR: Could not find a version that satisfies the requirement libvips (from versions: none)
ERROR: No matching distribution found for libvips


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image

model = AutoModelForCausalLM.from_pretrained(
    "vikhyatk/moondream2",
    revision="2025-04-14",
    trust_remote_code=True,
    # Uncomment to run on GPU.
    # device_map={"": "cuda"}
)

# Captioning
print("Short caption:")
print(model.caption(image, length="short")["caption"])

print("\nNormal caption:")
for t in model.caption(image, length="normal", stream=True)["caption"]:
    # Streaming generation example, supported for caption() and detect()
    print(t, end="", flush=True)
print(model.caption(image, length="normal"))

# Visual Querying
print("\nVisual query: 'How many people are in the image?'")
print(model.query(image, "How many people are in the image?")["answer"])

# Object Detection
print("\nObject detection: 'face'")
objects = model.detect(image, "face")["objects"]
print(f"Found {len(objects)} face(s)")

# Pointing
print("\nPointing: 'person'")
points = model.point(image, "person")["points"]
print(f"Found {len(points)} person(s)")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: cannot load library 'libvips.so.42': libvips.so.42: cannot open shared object file: No such file or directory.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libvips.so.42'

In [2]:
!pip install transformers==4.37.2 einops

  Using cached transformers-4.37.2-py3-none-any.whl.metadata (129 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.


In [1]:
# --- Imports ---
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from PIL import Image
import requests # For URL loading
from io import BytesIO # For URL loading
import time

# --- Configuration ---
# These should match the values you intend to use
model_id = "vikhyatk/moondream2"
revision = "2024-05-08"

# Determine device and model dtype
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    model_dtype = torch.float16 # Common for GPU inference with Moondream2
else:
    model_dtype = torch.float32 # Standard for CPU

print(f"Using device: {device}")
print(f"Using model dtype: {model_dtype}")

Using device: cuda
Using model dtype: torch.float16


In [3]:
# --- System Prompt for Full Text Extraction ---
system_prompt_ocr = """You are an expert Optical Character Recognition (OCR) assistant. Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image. You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image. Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant. If there is no text in the image, output an empty string.
"""
# --- Initialize Model and Tokenizer ---
print(f"Loading model: {model_id} (revision: {revision})")
model = None # Initialize model to None
tokenizer = None # Initialize tokenizer to None
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        revision=revision,
        torch_dtype=model_dtype
    ).to(device)
    model.eval()

    tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)
    print(f"Model and tokenizer loaded successfully on {device}.")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    exit() # Critical error, exit script

# --- Load Image ---
# Option 1: Load image from a local file path
image_source = "/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg"
is_url = False

# Option 2: Load image from a URL
# image_source = "https://raw.githubusercontent.com/vikhyat/moondream/main/assets/demo-2.jpg"
# is_url = True

print(f"Attempting to load image from: {image_source}")
image = None
try:
    if is_url:
        response = requests.get(image_source)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        print("Image loaded successfully from URL.")
    else:
        image = Image.open(image_source)
        print("Image loaded successfully from local file.")

except requests.exceptions.RequestException as e:
    print(f"Error fetching image from URL '{image_source}': {e}")
    exit()
except FileNotFoundError:
    print(f"Error: Image file not found at '{image_source}'.")
    exit()
except IOError as e:
    print(f"Error: Could not open or read image file from '{image_source}'. Details: {e}")
    exit()
except Exception as e:
    print(f"An unexpected error occurred while loading the image: {e}")
    exit()

if image is None:
    print("Critical error: Image could not be loaded. Exiting.")
    exit()

# --- Encode Image and Ask the OCR "Question" ---
print("\nEncoding image...")
image_encode_start_time = time.perf_counter()
try:
    # The check for model being None is good practice, though exit() should prevent this.
    if model is None or tokenizer is None:
        print("Critical error: Model or tokenizer not available. Exiting.")
        exit()

    enc_image = model.encode_image(image)
    image_encode_end_time = time.perf_counter()
    print("Image encoded.")
    image_encode_duration = image_encode_end_time - image_encode_start_time
    print(f"Image Encoding Time: {image_encode_duration:.4f} seconds")

    print(f"\nOCR Starting...")
    ocr_start_time = time.perf_counter()
    extracted_text = model.answer_question(
        enc_image,
        system_prompt_ocr,
        tokenizer
    )
    ocr_end_time = time.perf_counter()
    ocr_duration = ocr_end_time - ocr_start_time
    print(f"OCR (Question Answering) Time: {ocr_duration:.2f} seconds")

    print("\n--- Extracted Text ---")
    print(extracted_text)
    print("--- End of Extracted Text ---")

except Exception as e:
    print(f"An error occurred during image encoding or answering question: {e}")
    # For debugging, you might want to print the full traceback
    # import traceback
    # traceback.print_exc()

Loading model: vikhyatk/moondream2 (revision: 2024-05-08)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model and tokenizer loaded successfully on cuda.
Attempting to load image from: /content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg
Image loaded successfully from local file.

Encoding image...
Image encoded.
Image Encoding Time: 0.0514 seconds

OCR Starting...
OCR (Question Answering) Time: 1.94 seconds

--- Extracted Text ---
The California driver's license card, which is a document that serves as identification and proof of identity for drivers in California. The card features a photograph of a man, along with various fields for personal information and details. The task is to extract all the text, numbers, and symbols visible in the image, without adding any additional text or commentary.
--- End of Extracted Text ---


# Phi mini Vision

In [2]:
!pip install accelerate==0.30.0

  Using cached accelerate-0.30.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-0.30.0-py3-none-any.whl (302 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.6.0
    Uninstalling accelerate-1.6.0:
      Successfully uninstalled accelerate-1.6.0


In [1]:
!pip install numpy==1.24.4 requests==2.31.0 transformers==4.43.0 #torch==2.3.0 torchvision==0.18.0 accelerate==0.30.0 # flash_attn==2.5.8 Pillow==10.3.0 --no-build-isolation

  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached transformers-4.43.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached transformers-4.43.0-py3-none-any.whl (9.4 MB)
Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tokenize

In [4]:
!pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 57.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.


In [ ]:
from PIL import Image
import requests
from transformers import AutoModelForCausalLM, AutoProcessor
import torch # Explicitly import torch for torch_dtype

# Ensure you have a CUDA-enabled GPU available
if not torch.cuda.is_available():
    print("CUDA is not available. This script is intended to run on a CUDA-enabled GPU.")
    print("If you have a CUDA GPU, ensure PyTorch is installed with CUDA support.")
    exit()

model_id = "microsoft/Phi-3.5-vision-instruct"

# --- System Prompt for Summarization (Adjusted) ---
# Changed from OCR to better match the user task.
system_prompt_content = """You are a helpful AI assistant.
You will be provided with an image.
Your task is to describe and summarize the content of the image.
"""

print("Loading model... This may take a while.")
try:
    # --- FIX 1: Changed flash_attention_2 to eager ---
    # This will resolve the ImportError if flash-attn is not installed.
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="cuda",
        trust_remote_code=True,
        torch_dtype="auto",
        _attn_implementation='eager' # Use 'eager' if flash_attn is not installed/working
    )
    print("Model loaded successfully.")
except ImportError as e:
    print(f"ImportError during model loading: {e}")
    print("This usually means your 'transformers' library is outdated or a dependency is missing.")
    print("Please try: pip install --upgrade transformers accelerate")
    print("If specifically for flash_attention_2, ensure 'flash-attn' is installed: pip install flash-attn --no-build-isolation")
    print("Switched _attn_implementation to 'eager' as a fallback.")
    exit()
except Exception as e:
    print(f"An error occurred while loading the model: {e}")
    exit()


print("Loading processor...")
processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True,
)
print("Processor loaded successfully.")

images = []
image_placeholders = []

# --- FIX 2: Load only ONE image ---
image_file_path = "/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg"
num_images_to_load = 1 # Changed from 19

print(f"Attempting to load the image from '{image_file_path}' {num_images_to_load} time(s)...")
# Simplified loading for a single image
try:
    print(f"Loading image from {image_file_path}...")
    image = Image.open(image_file_path)
    images.append(image.copy())
    image_placeholders.append(f"<|image_{len(images)}|>") # Will be <|image_1|>
    print(f"Successfully loaded the image.")
except FileNotFoundError:
    print(f"Error: Image file not found at '{image_file_path}'.")
    print(f"Please ensure the file exists at this path in your environment (e.g., upload to Colab's /content/).")
    exit()
except IOError as e:
    print(f"Error: Could not open or read image from '{image_file_path}'. Error: {e}")
    exit()
except Exception as e:
    print(f"Error: An unexpected error occurred while loading the image. Error: {e}")
    exit()

if not images:
    print("No image was loaded. Exiting.")
    exit()

print(f"Successfully loaded {len(images)} image(s).")
# --- END OF FIXED IMAGE LOADING ---


# Construct the string of image placeholders
image_placeholders_str = "".join([f"{ph}\n" for ph in image_placeholders])

# Define the user's task instruction
# Adjusted task slightly for a single image
user_task_instruction = "Summarize the content of this image."

# Combine placeholders and task instruction for the user message content
user_content = image_placeholders_str + user_task_instruction

# Construct the messages list including the system prompt
messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user", "content": user_content},
]

print("Applying chat template and tokenizing...")
try:
    prompt = processor.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = processor(text=prompt, images=images, return_tensors="pt").to("cuda:0")
    print("Inputs prepared and moved to CUDA.")
except Exception as e:
    print(f"Error during input processing: {e}")
    exit()

generation_args = {
    "max_new_tokens": 500, # Reduced slightly, can be increased if needed
    "temperature": 0.0,
    "do_sample": False,
}

print("Generating response...")
try:
    generate_ids = model.generate(**inputs,
                                  eos_token_id=processor.tokenizer.eos_token_id,
                                  **generation_args
                                 )

    input_token_len = inputs['input_ids'].shape[1]
    generate_ids = generate_ids[:, input_token_len:]

    response = processor.batch_decode(generate_ids,
                                      skip_special_tokens=True,
                                      clean_up_tokenization_spaces=False)[0]

    print("\nGenerated Response:")
    print(response)

except torch.cuda.OutOfMemoryError:
    print("\nCUDA Out of Memory Error!")
    print("Try the following:")
    print("1. If you were loading many images previously, this should be less of an issue now.")
    print("2. Ensure 'eager' attention is working (it should be).")
    print("3. Close other applications using GPU memory.")
    print("4. Consider a GPU with more VRAM if consistently hitting this limit with complex tasks.")
except Exception as e:
    print(f"An error occurred during generation: {e}")

Loading model... This may take a while.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# استخراج النص من الصورة
text = model.chat(tokenizer, pixel_values, '<image>\nPlease Extract the text from the image.', generation_config)

# تقسيم النص إلى كلمات
words = text.split()

# تجميع الكلمات في مجموعات من خمس كلمات
lines = [' '.join(words[i:i+5]) for i in range(0, len(words), 5)]

# عرض النص
for line in lines:
    print(line)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Sure, here is the extracted
text from the image: ```
Patients with severe respiratory conditions,
such as severe asthma, acute
bronchitis, or chronic obstructive pulmonary
disease, may have difficulty traveling
by air. Flying requires the
ability to adapt to low
oxygen levels and dry air,
which can be a challenge
for people with respiratory problems.
In rare cases, flights can
be risky even for individuals
with non-incurable respiratory problems if
they do not have adequate
treatment or control of their
condition. It is important to
consult a doctor before traveling
and to ensure that you
carry the necessary medications and
follow the flight recommendations for
your condition. ```


In [ ]:
!pip install -U bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.4 MB/s eta 0:00:00


In [ ]:
!pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 72.2 MB/s eta 0:00:00


In [ ]:


import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer


# Load the model and tokenizer
path = "OpenGVLab/InternVL2_5-1B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=True,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True
).eval()  # Remove .cuda() here

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/9.02k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.38M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

In [ ]:
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

# Define build_transform and dynamic_preprocess functions
def build_transform(input_size=448):
    """Builds a transformation pipeline for image preprocessing."""
    transform = T.Compose([
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform


def dynamic_preprocess(image, image_size=448, use_thumbnail=True, max_num=12):
    """Dynamically preprocesses the image."""
    # Placeholder for dynamic preprocessing logic (replace with actual implementation)
    # This example simply returns the original image in a list
    return [image]

# Load the model and tokenizer
path = "OpenGVLab/InternVL2_5-1B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=True,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True
).eval()  # Remove .cuda() here

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)


def load_image(image_file, input_size=448, max_num=None):  # Parameter max_num set to None.
    """Loads an image and preprocesses it.
    If 'max_num' is not None, it is passed to dynamic_preprocess, otherwise a default of 12 is used."""

    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    if max_num:  # Check if max_num is provided
        images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    else:
        images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=12) # Default max_num to 12
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
image_path = '/content/1.png'
def load_image(image_file, input_size=448, max_num=None):  # Parameter max_num set to None.
    """Loads an image and preprocesses it.
    If 'max_num' is not None, it is passed to dynamic_preprocess, otherwise a default of 12 is used."""

    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    if max_num:  # Check if max_num is provided
        images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    else:
        images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=12) # Default max_num to 12
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

pixel_values = load_image(image_path).to(torch.bfloat16) # Keep pixel_values on CPU
generation_config = dict(max_new_tokens=1024, do_sample=True)

In [ ]:
question = '<image>\nPlease describe the image shortly.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


User: <image>
Please describe the image shortly.
Assistant: The image is an excerpt from a document discussing travel safety and medical considerations for people with severe respiratory conditions. It warns about the difficulty of adapting to low oxygen levels and advises consulting a doctor before flying if necessary to ensure appropriate treatments.


In [ ]:
question = '<image>\nPlease extract text from image .'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


User: <image>
Please extract text from image .
Assistant: Patients with severe respiratory conditions, such as severe asthma, acute bronchitis, or chronic obstructive pulmonary disease, may have difficulty traveling by air due to the ability to adapt to low oxygen levels and dry air, which can be a challenge for people with respiratory problems. In rare cases, flights can be risky even for individuals with non-irritable respiratory problems if they do not have adequate treatment or control of their condition. It is important to consult a doctor before traveling and to ensure that you carry the necessary medications and follow the flight recommendations for your condition.


In [ ]:
question = '<image>\nPlease Extract the text from the image and make 10 words in each line. After the 10 words in the first line, complete the next line until the extracted text is finished.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


User: <image>
Please Extract the text from the image and make 10 words in each line. After the 10 words in the first line, complete the next line until the extracted text is finished.
Assistant: Here is the extracted text:

- Patients with severe respiratory conditions may have difficulty traveling by air.
- Low oxygen levels can be challenging, especially for those with chronic obstructive pulmonary disease.
- Some travelers risk air discomfort or injury.
- Individuals with non-incureable respiratory problems should consult doctors.
- Medications need a physician's approval before use.
- Follow guide for condition's recommendations during travel.

Complete text extraction line:
- Patients with severe respiratory conditions may have difficulty traveling by air.
- Low oxygen levels can be challenging, especially for those with chronic obstructive pulmonary disease.
- Some travelers risk air discomfort or injury.
- Individuals with non-incureable respiratory problems should consult physi

In [ ]:
# single-image single-round conversation
question = '<image>\nPlease Extract the text from the image.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')

In [ ]:
# استخراج النص من الصورة
text = model.chat(tokenizer, pixel_values, '<image>\nPlease Extract the text from the image.', generation_config)

# تقسيم النص إلى كلمات
words = text.split()

# تجميع الكلمات في مجموعات من خمس كلمات
lines = [' '.join(words[i:i+5]) for i in range(0, len(words), 5)]

# عرض النص
for line in lines:
    print(line)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Sure, here is the extracted
text from the image: ```
Patients with severe respiratory conditions,
such as severe asthma, acute
bronchitis, or chronic obstructive pulmonary
disease, may have difficulty traveling
by air. Flying requires the
ability to adapt to low
oxygen levels and dry air,
which can be a challenge
for people with respiratory problems.
In rare cases, flights can
be risky even for individuals
with non-incurable respiratory problems if
they do not have adequate
treatment or control of their
condition. It is important to
consult a doctor before traveling
and to ensure that you
carry the necessary medications and
follow the flight recommendations for
your condition. ```
